In [1]:
#default_exp engines.detectron2.augmentations
#all_examples

# Data augmentation for detectron2 models

In [2]:
#export
from drone_detector.imports import *
from drone_detector.utils import *

import detectron2
from detectron2.data import transforms as T
from fvcore.transforms import BlendTransform, VFlipTransform


In [4]:
from fvcore.transforms import HFlipTransform

In [9]:
# export

def VFlip_rotated_box(transform, rotated_boxes):
    """
    Apply the vertical flip transform on rotated boxes.
    
    Args:
        rotated_boxes (ndarray): Nx5 floating point array of
            (x_center, y_center, width, height, angle_degrees) format
            in absolute coodinates
    """
    # Transform y_center
    rotated_boxes[:,1] = transform.height - rotated_boxes[:,1]
    # Transform angle
    rotated_boxes[:, 4] = -rotated_boxes[:,4]
    return rotated_boxes


BlendTransform.register_type('rotated_box', lambda t, x: x)
VFlipTransform.register_type('rotated_box', VFlip_rotated_box)

@patch_to(T.RotationTransform)
def apply_rotated_box(self, rotated_boxes):
    """
    rotated_boxes should be a N*5 array-like, containing N couples of
    (x_center, y_center, width, height, angle) boxes
    Most likely wont work with expand=True
    """
    if len(rotated_boxes) == 0 or self.angle % 360 == 0:
        return rotated_boxes
    center_coords = rotated_boxes[:,:2]
    rotated_boxes[:,:2] = self.apply_coords(center_coords)
    rotated_boxes[:,4] += self.angle
    return rotated_boxes

In [13]:
# export

def build_aug_transforms(cfg:detectron2.config.CfgNode, flip_horiz:bool=True, flip_vert:bool=False, max_rotate:int=10,
                         brightness_limits:Tuple[int,int]=(0.8,1.4), contrast_limits:Tuple[int,int]=(0.8,1.4), 
                         saturation_limits:Tuple[int,int]=(0.8,1.4), p_lighting:float=0.75
                         ) -> detectron2.data.transforms.AugmentationList:
    "Build a list of detectron2 augmentations"
    augs = []
    augs.append(T.ResizeShortestEdge(cfg.INPUT.MIN_SIZE_TRAIN, 
                                     cfg.INPUT.MAX_SIZE_TRAIN, 
                                     cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING))
    if flip_horiz: augs.append(T.RandomFlip(prob=0.5, horizontal=True, vertical=False))
    if flip_vert: augs.append(T.RandomFlip(prob=0.5, horizontal=False, vertical=True))
    if max_rotate: augs.append(T.RandomRotation(angle=[-max_rotate, max_rotate], expand=False))
    if brightness_limits: augs.append(T.RandomApply(prob=p_lighting, tfm_or_aug=T.RandomBrightness(*brightness_limits)))
    if contrast_limits: augs.append(T.RandomApply(prob=p_lighting, tfm_or_aug=T.RandomContrast(*contrast_limits)))
    if saturation_limits: augs.append(T.RandomApply(prob=p_lighting, tfm_or_aug=T.RandomSaturation(*saturation_limits)))
    
    return augs